In [ ]:
!pip install datasets transformers huggingface_hub transformers[torch] accelerate --upgrade

import re
from sklearn.model_selection import train_test_split

f = open("./india_at_olympics.txt", "r")
text = f.readlines()

def build_text_files(data_text, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for texts in data_text:
        summary = str(texts).strip()
        summary = re.sub(r"\s", " ", summary)
        data += summary + "  "
    f.write(data)

train, test = train_test_split(text,test_size=0.15)


build_text_files(train,'train_dataset.txt')
build_text_files(test,'test_dataset.txt')

print("Train dataset length: "+str(len(train)))
print("Test dataset length: "+ str(len(test)))

print(train)
print(test)

train_path = "train_dataset.txt"
test_path = "test_dataset.txt"

from transformers import TextDataset, DataCollatorForLanguageModeling

def load_dataset(train_path, test_path, tokenizer):
  train_dataset = TextDataset(tokenizer=tokenizer, file_path=train_path, block_size=64)

  test_dataset = TextDataset(tokenizer=tokenizer, file_path=test_path, block_size=64)

  data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

  return train_dataset, test_dataset, data_collator

train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokeinzer)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 M

In [ ]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import torch

# Load your models
model_v1 = AutoModelForSequenceClassification.from_pretrained("gpt2")
model_v2 = AutoModelForSequenceClassification.from_pretrained("gpt2-wiki-india-at-olympics")



In [ ]:
def evaluate_model(model, dataset):
    trainer = Trainer(
        model=model,
        eval_dataset=dataset,
    )
    eval_results = trainer.evaluate()
    return eval_results

# Evaluate model_v1
eval_results_v1 = evaluate_model(model_v1, test_dataset)
print("Evaluation results for model_v1:")
print(eval_results_v1)

# Evaluate model_v2
eval_results_v2 = evaluate_model(model_v2, test_dataset)
print("Evaluation results for model_v2:")
print(eval_results_v2)
